In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import math
from sklearn.model_selection import train_test_split
print(f"Tensorflow version: \t{tf.__version__}")
print(f"Keras version: \t\t\t{keras.__version__}")

Tensorflow version: 	2.20.0
Keras version: 			3.11.3


In [2]:
# Load data

images = np.load("../A1_data_75/images.npy")
labels = np.load("../A1_data_75/labels.npy")

In [3]:
# Process to segment into train, test, and validation sets
images = images / 255.0

X_train, X_temp, y_train, y_temp = train_test_split(images,labels,
                                         random_state=19,
                                         test_size=0.2,
                                         shuffle=True)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp,
                                       test_size=0.5,
                                       shuffle=True)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(14400, 75, 75) (14400, 2)
(1800, 75, 75) (1800, 2)
(1800, 75, 75) (1800, 2)


In [4]:
def class_convert(y, interval=30):
    vals = []

    for hr, min in y:
        # Assigns a value in range 0, 23 for each hour minute combination, resulting in 24*60/interval classes. 0 for 0-30 min, 1 for 30 min-1 hr for interval=30, etc.
        classification_val = math.floor(min/interval + hr*60/interval)
        vals.append(classification_val)
        # print(f"Actual value: {hr} hours, {min} minutes")
        # print(f"Corresponding label for classification: {classification_val}")
    return vals

conv_y_test = class_convert(y_test)